In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(598, 2881)
(310, 2879)


In [4]:
# 모두 결측치인 변수 제거
def remove_all_nan(dataframe):
  col_list = dataframe.columns
  nan_list = []
  nan_cnt = []
  nan_col = []
  full_list = []

  for col in col_list:
    if dataframe[col].isnull().sum() == 0:
      full_list.append(col)
      continue
    nan_list.append([col,dataframe[col].isnull().sum()])
    nan_cnt.append(dataframe[col].isnull().sum())
    nan_col.append(col)

  del_col = []
  for nan in nan_list:
    if nan[1] == len(dataframe):
      del_col.append(nan[0])
  
  return dataframe.drop(columns=del_col)

In [5]:
'''값이 1개 존재하면 제거'''

def remove_one_value(dataframe):
  del_col = []
  col_list = dataframe.columns
  for col in col_list:
      if dataframe[col].nunique()==1 :
          del_col.append(col)
  
  return dataframe.drop(columns=del_col)

In [6]:
def ctgan_model(dataframe):
  discrete = remove_all_nan(dataframe)
  discrete = remove_one_value(discrete)
  discrete = discrete.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
  discrete_columns = discrete.columns

  real_data = discrete.copy()

  ctgan = CTGAN(epochs=50)
  ctgan.fit(real_data, discrete_columns)

  return real_data, ctgan

In [7]:
!pip install --target=$my_path ctgan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 KB 625.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
  Attempting uninstall: packaging
    Found existing installation: packaging 23.0
    Uninstalling packaging-23.0:
      Successfully uninstalled packaging-23.0


In [8]:
!pip install --target=$my_path numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from ctgan import CTGAN
from ctgan import load_demo

# LINE별 dataset 구축

In [10]:
train_T050304 = train_df[train_df['LINE'] == 'T050304']
train_T050307 = train_df[train_df['LINE'] == 'T050307']
train_T100304 = train_df[train_df['LINE'] == 'T100304']
train_T100306 = train_df[train_df['LINE'] == 'T100306']
train_T010306 = train_df[train_df['LINE'] == 'T010306']
train_T010305 = train_df[train_df['LINE'] == 'T010305']

In [11]:
test_T050304 = test_df[test_df['LINE'] == 'T050304']
test_T050307 = test_df[test_df['LINE'] == 'T050307']
test_T100304 = test_df[test_df['LINE'] == 'T100304']
test_T100306 = test_df[test_df['LINE'] == 'T100306']
test_T010306 = test_df[test_df['LINE'] == 'T010306']
test_T010305 = test_df[test_df['LINE'] == 'T010305']

test set에 존재하는 모두 nan인 변수에 대해 -1로 처리하는 것과 없애는 것 중 뭐가 좋은지는 논의해봐야할 것 같습니다.
  - 일단 없애지않고 진행하겠습니다.

In [12]:
print('train T050304 :' ,train_T050304['PRODUCT_CODE'].unique(), 'test T050304 :' ,test_T050304['PRODUCT_CODE'].unique())
print('train T050307 :' ,train_T050307['PRODUCT_CODE'].unique(), 'test T050307 :' ,test_T050307['PRODUCT_CODE'].unique())
print('train T100304 :' ,train_T100304['PRODUCT_CODE'].unique(), 'test T100304 :' ,test_T100304['PRODUCT_CODE'].unique())
print('train T100306 :' ,train_T100306['PRODUCT_CODE'].unique(), 'test T100306 :' ,test_T100306['PRODUCT_CODE'].unique())
print('train T010306 :' ,train_T010306['PRODUCT_CODE'].unique(), 'test T010306 :' ,test_T010306['PRODUCT_CODE'].unique())
print('train T010305 :' ,train_T010305['PRODUCT_CODE'].unique(), 'test T010305 :' ,test_T010305['PRODUCT_CODE'].unique())


train T050304 : ['A_31'] test T050304 : ['A_31']
train T050307 : ['A_31'] test T050307 : ['A_31']
train T100304 : ['T_31' 'O_31'] test T100304 : ['T_31' 'O_31']
train T100306 : ['T_31' 'O_31'] test T100306 : ['T_31' 'O_31']
train T010306 : ['A_31'] test T010306 : ['A_31']
train T010305 : ['A_31'] test T010305 : ['A_31']


In [13]:
# A 생성 LINE
train_T050304, test_T050304
train_T050307, test_T050307
train_T010306, test_T010306
train_T010305, test_T010305

# T, O 생성 LINE
train_T100304, test_T100304
train_T100306, test_T100306

(    PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
 26   TRAIN_026        1   0.531992   2022-06-19 9:20  T100306         T_31   
 30   TRAIN_030        1   0.528646  2022-06-19 23:39  T100306         T_31   
 36   TRAIN_036        1   0.529633   2022-06-22 3:39  T100306         T_31   
 42   TRAIN_042        1   0.530965   2022-06-23 3:39  T100306         T_31   
 43   TRAIN_043        1   0.529498   2022-06-23 6:28  T100306         T_31   
 ..         ...      ...        ...               ...      ...          ...   
 589  TRAIN_589        1   0.529510  2022-09-06 18:00  T100306         T_31   
 590  TRAIN_590        1   0.529948   2022-09-07 1:01  T100306         T_31   
 591  TRAIN_591        1   0.529308   2022-09-07 1:09  T100306         T_31   
 593  TRAIN_593        1   0.526546  2022-09-08 14:30  T100306         T_31   
 597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   
 
       X_1   X_2  X_3   X_4  ...  X_2866  X_2867  

## LINE별 train set은 최대한 구축 = 모두 NaN이 아니라면 -1 * max()을 적용
  - 단 y_class별로 나눠서 적용하는게 합리적인듯
  - aug(증폭 몇배수) = 10

In [14]:
aug=10

PRODUCT_CODE A

Y_Class == 0

In [15]:
train_T050304_0 = train_T050304[train_T050304['Y_Class'] == 0]
train_T050304_0 = train_T050304_0.fillna(-1 * train_T050304_0.max())
train_T050304_0

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
34,TRAIN_034,0,0.521524,2022-06-21 17:36,T050304,A_31,NaN,NaN,NaN,NaN,...,55.03,52.24,55.33,57.49,67.31,1.0,NaN,NaN,NaN,NaN
109,TRAIN_109,0,0.516711,2022-07-06 19:15,T050304,A_31,NaN,NaN,NaN,NaN,...,56.86,50.46,40.49,57.34,66.37,1.0,NaN,NaN,NaN,NaN
110,TRAIN_110,0,0.519451,2022-07-06 19:23,T050304,A_31,NaN,NaN,NaN,NaN,...,48.28,51.25,55.43,56.95,67.44,1.0,NaN,NaN,NaN,NaN
111,TRAIN_111,0,0.518960,2022-07-06 20:41,T050304,A_31,NaN,NaN,NaN,NaN,...,-60.24,-62.37,-59.68,-59.93,-67.44,1.0,NaN,NaN,NaN,NaN
131,TRAIN_131,0,0.519810,2022-07-10 0:57,T050304,A_31,NaN,NaN,NaN,NaN,...,52.97,59.15,42.29,57.22,65.46,1.0,NaN,NaN,NaN,NaN
132,TRAIN_132,0,0.521259,2022-07-10 1:05,T050304,A_31,NaN,NaN,NaN,NaN,...,-60.24,-62.37,-59.68,-59.93,-67.44,1.0,NaN,NaN,NaN,NaN
202,TRAIN_202,0,0.521544,2022-07-21 4:53,T050304,A_31,NaN,NaN,NaN,NaN,...,-60.24,-62.37,-59.68,-59.93,-67.44,1.0,NaN,NaN,NaN,NaN
255,TRAIN_255,0,0.515083,2022-07-25 12:39,T050304,A_31,NaN,NaN,NaN,NaN,...,60.24,56.16,54.52,56.42,66.85,1.0,NaN,NaN,NaN,NaN
322,TRAIN_322,0,0.522300,2022-07-31 22:37,T050304,A_31,NaN,NaN,NaN,NaN,...,55.99,52.08,40.90,55.31,62.80,1.0,NaN,NaN,NaN,NaN
344,TRAIN_344,0,0.524567,2022-08-03 8:28,T050304,A_31,NaN,NaN,NaN,NaN,...,54.63,59.05,40.77,55.24,64.69,1.0,NaN,NaN,NaN,NaN


In [16]:
real_data, ctgan = ctgan_model(train_T050304_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T050304'
total_data['Y_Class'] = 0
train_T050304_0 = total_data.copy()

<ipython-input-16-970604473b23>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T050304'
<ipython-input-16-970604473b23>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


Y_Class == 1

In [17]:
train_T050304_1 = train_T050304[train_T050304['Y_Class'] == 1]
train_T050304_1 = train_T050304_1.fillna(-1 * train_T050304_1.max())
train_T050304_1

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,-1.0,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,-1.0,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,-1.0,NaN,NaN,NaN,NaN
6,TRAIN_006,1,0.533665,2022-06-13 6:03,T050304,A_31,NaN,NaN,NaN,NaN,...,32.50,41.42,38.36,30.83,76.93,-1.0,NaN,NaN,NaN,NaN
8,TRAIN_008,1,0.531821,2022-06-13 6:19,T050304,A_31,NaN,NaN,NaN,NaN,...,42.25,43.17,55.60,33.26,78.50,-1.0,NaN,NaN,NaN,NaN
10,TRAIN_010,1,0.531503,2022-06-13 6:36,T050304,A_31,NaN,NaN,NaN,NaN,...,32.12,34.74,54.93,33.16,76.30,-1.0,NaN,NaN,NaN,NaN
12,TRAIN_012,1,0.533632,2022-06-13 6:52,T050304,A_31,NaN,NaN,NaN,NaN,...,36.55,32.32,54.25,35.15,78.86,-1.0,NaN,NaN,NaN,NaN
14,TRAIN_014,1,0.531254,2022-06-13 7:08,T050304,A_31,NaN,NaN,NaN,NaN,...,40.42,38.33,40.14,30.72,76.52,-1.0,NaN,NaN,NaN,NaN
16,TRAIN_016,1,0.534054,2022-06-13 7:25,T050304,A_31,NaN,NaN,NaN,NaN,...,40.53,31.70,49.49,35.73,79.75,-1.0,NaN,NaN,NaN,NaN
18,TRAIN_018,1,0.532170,2022-06-13 7:41,T050304,A_31,NaN,NaN,NaN,NaN,...,35.07,48.38,36.24,33.00,75.82,-1.0,NaN,NaN,NaN,NaN


Y_Class == 2

In [18]:
train_T050304_2 = train_T050304[train_T050304['Y_Class'] == 2]
train_T050304_2 = train_T050304_2.fillna(-1 * train_T050304_2.max())
train_T050304_2

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
21,TRAIN_021,2,0.544129,2022-06-14 5:58,T050304,A_31,NaN,NaN,NaN,NaN,...,-57.94,-63.42,-60.00,-58.18,-67.22,1.0,NaN,NaN,NaN,NaN
79,TRAIN_079,2,0.538629,2022-07-01 0:54,T050304,A_31,NaN,NaN,NaN,NaN,...,53.49,55.59,60.00,58.18,66.79,1.0,NaN,NaN,NaN,NaN
80,TRAIN_080,2,0.537733,2022-07-01 1:11,T050304,A_31,NaN,NaN,NaN,NaN,...,55.89,63.42,42.71,57.03,66.20,1.0,NaN,NaN,NaN,NaN
188,TRAIN_188,2,0.535405,2022-07-19 6:27,T050304,A_31,NaN,NaN,NaN,NaN,...,-57.94,-63.42,-60.00,-58.18,-67.22,1.0,NaN,NaN,NaN,NaN
314,TRAIN_314,2,0.537046,2022-07-31 14:15,T050304,A_31,NaN,NaN,NaN,NaN,...,53.49,58.06,33.43,54.71,61.93,1.0,NaN,NaN,NaN,NaN
315,TRAIN_315,2,0.535965,2022-07-31 14:53,T050304,A_31,NaN,NaN,NaN,NaN,...,56.71,61.84,40.60,56.59,65.12,1.0,NaN,NaN,NaN,NaN
556,TRAIN_556,2,0.560568,2022-08-28 2:54,T050304,A_31,NaN,NaN,NaN,NaN,...,53.49,58.76,42.22,57.92,66.18,1.0,NaN,NaN,NaN,NaN
557,TRAIN_557,2,0.559463,2022-08-28 3:03,T050304,A_31,NaN,NaN,NaN,NaN,...,50.87,57.13,53.07,56.21,67.22,1.0,NaN,NaN,NaN,NaN
562,TRAIN_562,2,0.541429,2022-09-03 1:14,T050304,A_31,NaN,NaN,NaN,NaN,...,56.38,57.10,51.49,56.08,64.40,1.0,NaN,NaN,NaN,NaN
563,TRAIN_563,2,0.537465,2022-09-03 1:22,T050304,A_31,NaN,NaN,NaN,NaN,...,49.92,54.66,50.56,48.63,62.17,1.0,NaN,NaN,NaN,NaN


In [19]:
train_T050304= pd.concat([train_T050304_0,train_T050304_1,train_T050304_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T050304.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T050304 = train_T050304[key_cols]
train_T050304

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_129,X_132,X_133,...,X_2824,X_2826,X_2827,X_2838,X_2844,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,7813.0,7813.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,7815.0,7815.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,7817.0,7817.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
6,TRAIN_006,1,0.533665,2022-06-13 6:03,T050304,A_31,7820.0,7820.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
8,TRAIN_008,1,0.531821,2022-06-13 6:19,T050304,A_31,7822.0,7822.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,0,0.520252,NaN,T050304,NaN,24875.0,5539.0,0.20,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,NaN,0,0.515083,NaN,T050304,NaN,14813.0,24875.0,0.19,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,NaN,0,0.519451,NaN,T050304,NaN,6065.0,17441.0,0.19,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,NaN,0,0.517494,NaN,T050304,NaN,6065.0,14813.0,0.19,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
train_T050307_0 = train_T050307[train_T050307['Y_Class'] == 0]
train_T050307_0 = train_T050307_0.fillna(train_T050307_0.max() * -1)

train_T050307_1 = train_T050307[train_T050307['Y_Class'] == 1]
train_T050307_1 = train_T050307_1.fillna(train_T050307_1.max() * -1)

train_T050307_2 = train_T050307[train_T050307['Y_Class'] == 2]
train_T050307_2 = train_T050307_2.fillna(train_T050307_2.max() * -1)

# train_T050307= pd.concat([train_T050307_0,train_T050307_1,train_T050307_2]).sort_values(by='PRODUCT_ID')

In [21]:
real_data, ctgan = ctgan_model(train_T050307_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T050307'
total_data['Y_Class'] = 0
train_T050307_0 = total_data.copy()

train_T050307= pd.concat([train_T050307_0,train_T050307_1,train_T050307_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T050307.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T050307 = train_T050307[key_cols]
train_T050307

<ipython-input-21-4083d8c7014c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T050307'
<ipython-input-21-4083d8c7014c>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_130,X_131,X_132,X_133,...,X_2790,X_2793,X_2795,X_2838,X_2844,X_2871,X_2872,X_2873,X_2874,X_2875
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,19854.0,19854.0,0.20,0.21,...,24.0,25.0,23.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,19856.0,19856.0,0.20,0.21,...,24.0,25.0,23.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
5,TRAIN_005,2,0.537833,2022-06-13 5:55,T050307,A_31,19859.0,19859.0,0.20,0.21,...,25.0,25.0,23.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
7,TRAIN_007,2,0.540003,2022-06-13 6:11,T050307,A_31,19861.0,19861.0,0.20,0.21,...,24.0,25.0,23.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
9,TRAIN_009,2,0.538049,2022-06-13 6:28,T050307,A_31,19864.0,19864.0,0.20,0.21,...,25.0,25.0,23.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,0,0.521406,NaN,T050307,NaN,13975.0,1102.0,0.19,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,0,0.521406,NaN,T050307,NaN,1055.0,1055.0,0.19,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,0,0.515884,NaN,T050307,NaN,1055.0,2540.0,0.19,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,0,0.519406,NaN,T050307,NaN,829.0,2540.0,0.20,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
train_T010306_0 = train_T010306[train_T010306['Y_Class'] == 0]
train_T010306_0 = train_T010306_0.fillna(train_T010306_0.max() * -1)

train_T010306_1 = train_T010306[train_T010306['Y_Class'] == 1]
train_T010306_1 = train_T010306_1.fillna(train_T010306_1.max() * -1)

train_T010306_2 = train_T010306[train_T010306['Y_Class'] == 2]
train_T010306_2 = train_T010306_2.fillna(train_T010306_2.max() * -1)

# train_T010306= pd.concat([train_T010306_0,train_T010306_1,train_T010306_2]).sort_values(by='PRODUCT_ID')

In [23]:
real_data, ctgan = ctgan_model(train_T010306_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T010306'
total_data['Y_Class'] = 0
train_T010306_0 = total_data.copy()

train_T010306= pd.concat([train_T010306_0,train_T010306_1,train_T010306_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T010306.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T010306 = train_T010306[key_cols]
train_T010306

<ipython-input-23-d04807ab160f>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T010306'
<ipython-input-23-d04807ab160f>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_247,X_248,X_256,X_257,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
27,TRAIN_027,2,0.540002,2022-06-19 20:18,T010306,A_31,87.0,144.0,500.8,509.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,TRAIN_031,2,0.536206,2022-06-20 21:55,T010306,A_31,93.0,150.0,498.7,507.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,TRAIN_032,2,0.548592,2022-06-21 7:06,T010306,A_31,89.0,152.0,497.7,505.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,TRAIN_038,2,0.536616,2022-06-22 4:37,T010306,A_31,91.0,158.0,500.7,508.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,TRAIN_047,1,0.533230,2022-06-23 15:42,T010306,A_31,95.0,166.0,501.2,509.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,NaN,0,0.524487,NaN,T010306,NaN,89.0,7.0,471.1,472.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,NaN,0,0.524487,NaN,T010306,NaN,91.0,24.0,471.1,472.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,NaN,0,0.515214,NaN,T010306,NaN,91.0,63.0,443.7,472.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,NaN,0,0.513135,NaN,T010306,NaN,89.0,1.0,480.5,485.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
train_T010305_0 = train_T010305[train_T010305['Y_Class'] == 0]
train_T010305_0 = train_T010305_0.fillna(train_T010305_0.max() * -1)

train_T010305_1 = train_T010305[train_T010305['Y_Class'] == 1]
train_T010305_1 = train_T010305_1.fillna(train_T010305_1.max() * -1)

train_T010305_2 = train_T010305[train_T010305['Y_Class'] == 2]
train_T010305_2 = train_T010305_2.fillna(train_T010305_2.max() * -1)

# train_T010305= pd.concat([train_T010305_0,train_T010305_1,train_T010305_2]).sort_values(by='PRODUCT_ID')

In [25]:
real_data, ctgan = ctgan_model(train_T010305_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T010305'
total_data['Y_Class'] = 0
train_T010305_0 = total_data.copy()

train_T010305= pd.concat([train_T010305_0,train_T010305_1,train_T010305_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T010305.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T010305 = train_T010305[key_cols]
train_T010305

<ipython-input-25-69d24ce57e1c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T010305'
<ipython-input-25-69d24ce57e1c>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_246,X_247,X_251,X_253,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
33,TRAIN_033,1,0.526656,2022-06-21 7:14,T010305,A_31,1.0,95.0,98.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,TRAIN_039,1,0.531583,2022-06-22 4:45,T010305,A_31,1.0,95.0,102.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,TRAIN_040,1,0.530829,2022-06-22 12:26,T010305,A_31,1.0,95.0,105.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,TRAIN_041,1,0.528270,2022-06-22 12:34,T010305,A_31,1.0,87.0,105.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,TRAIN_048,1,0.533273,2022-06-23 15:50,T010305,A_31,1.0,91.0,111.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NaN,0,0.523173,NaN,T010305,NaN,2.0,91.0,127.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,0,0.500856,NaN,T010305,NaN,1.0,89.0,122.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,0,0.521249,NaN,T010305,NaN,2.0,89.0,90.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,NaN,0,0.518024,NaN,T010305,NaN,1.0,88.0,67.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
trainA_31 = pd.concat([train_T050304,train_T050307,train_T010306,train_T010305]).sort_values(by='PRODUCT_ID')
trainA_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_129,X_132,X_133,...,X_2824,X_2826,X_2827,X_2838,X_2844,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,7813.0,7813.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,0.20,0.21,...,61.0,53.8,53.8,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,7815.0,7815.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,0.20,0.21,...,61.0,53.8,53.8,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,7817.0,7817.0,0.19,0.20,...,55.2,52.6,52.6,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NaN,0,0.523173,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,0,0.500856,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,0,0.521249,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,NaN,0,0.518024,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


PRODUCT_CODE = T

In [27]:
train_T100304[train_T100304['PRODUCT_CODE'] == 'T_31']
train_T100304[train_T100304['PRODUCT_CODE'] == 'O_31']

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
569,TRAIN_569,1,0.530533,2022-09-03 18:32,T100304,O_31,4.0,98.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,TRAIN_571,1,0.525916,2022-09-03 18:48,T100304,O_31,4.0,100.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
train_T100306[train_T100306['PRODUCT_CODE'] == 'O_31']

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
570,TRAIN_570,2,0.534951,2022-09-03 18:40,T100306,O_31,6.0,90.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,TRAIN_572,2,0.535205,2022-09-03 18:56,T100306,O_31,6.0,89.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,TRAIN_597,1,0.533702,2022-09-08 14:46,T100306,O_31,21.0,87.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
train_T100304_T = train_T100304[train_T100304['PRODUCT_CODE'] == 'T_31']

In [30]:
train_T100304_T_0 = train_T100304_T[train_T100304_T['Y_Class'] == 0]
train_T100304_T_0 = train_T100304_T_0.fillna(train_T100304_T_0.max() * -1)

train_T100304_T_1 = train_T100304_T[train_T100304_T['Y_Class'] == 1]
train_T100304_T_1 = train_T100304_T_1.fillna(train_T100304_T_1.max() * -1)

train_T100304_T_2 = train_T100304_T[train_T100304_T['Y_Class'] == 2]
train_T100304_T_2 = train_T100304_T_2.fillna(train_T100304_T_2.max() * -1)

# train_T100304_T = pd.concat([train_T100304_T_0,train_T100304_T_1,train_T100304_T_2]).sort_values(by='PRODUCT_ID')

In [31]:
real_data, ctgan = ctgan_model(train_T100304_T_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T100304'
total_data['Y_Class'] = 0
train_T100304_T_0 = total_data.copy()

train_T100304_T= pd.concat([train_T100304_T_0,train_T100304_T_1,train_T100304_T_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T100304_T.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T100304_T = train_T100304_T[key_cols]
train_T100304_T

<ipython-input-31-fb19a7ada820>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T100304'
<ipython-input-31-fb19a7ada820>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_11,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
25,TRAIN_025,1,0.529362,2022-06-19 9:11,T100304,T_31,2.0,97.0,11.0,517.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,TRAIN_029,1,0.532405,2022-06-19 23:31,T100304,T_31,2.0,100.0,11.0,516.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,TRAIN_037,1,0.533110,2022-06-22 3:47,T100304,T_31,2.0,100.0,11.0,517.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,TRAIN_045,1,0.530948,2022-06-23 10:24,T100304,T_31,2.0,97.0,11.0,516.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,TRAIN_046,1,0.532219,2022-06-23 10:32,T100304,T_31,2.0,99.0,11.0,516.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,NaN,0,0.517463,NaN,T100304,NaN,2.0,101.0,11.0,494.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,0,0.507571,NaN,T100304,NaN,2.0,101.0,10.0,476.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,NaN,0,0.524862,NaN,T100304,NaN,2.0,100.0,10.0,514.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,NaN,0,0.517719,NaN,T100304,NaN,2.0,98.0,10.0,520.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
train_T100306_T = train_T100306[train_T100306['PRODUCT_CODE'] == 'T_31']

In [33]:
train_T100306_T_0 = train_T100306_T[train_T100306_T['Y_Class'] == 0]
train_T100306_T_0 = train_T100306_T_0.fillna(train_T100306_T_0.max() * -1)

train_T100306_T_1 = train_T100306_T[train_T100306_T['Y_Class'] == 1]
train_T100306_T_1 = train_T100306_T_1.fillna(train_T100306_T_1.max() * -1)

train_T100306_T_2 = train_T100306_T[train_T100306_T['Y_Class'] == 2]
train_T100306_T_2 = train_T100306_T_2.fillna(train_T100306_T_2.max() * -1)

# train_T100306_T = pd.concat([train_T100306_T_0,train_T100306_T_1,train_T100306_T_2]).sort_values(by='PRODUCT_ID')

In [34]:
real_data, ctgan = ctgan_model(train_T100306_T_0)
synthetic_data = ctgan.sample(real_data.shape[0] * aug)

total_data = pd.concat([real_data, synthetic_data])
total_data['LINE'] = 'T100306'
total_data['Y_Class'] = 0
train_T100306_T_0 = total_data.copy()

train_T100306_T= pd.concat([train_T100306_T_0,train_T100306_T_1,train_T100306_T_2]).sort_values(by='PRODUCT_ID')
key_cols = ['PRODUCT_ID', 'Y_Class', 'Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']
rest_cols = [i for i in train_T100306_T.columns if i not in key_cols]
key_cols.extend(rest_cols)
train_T100306_T = train_T100306_T[key_cols]
train_T100306_T

<ipython-input-34-e01d7e5365e9>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['LINE'] = 'T100306'
<ipython-input-34-e01d7e5365e9>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  total_data['Y_Class'] = 0


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_7,X_11,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
26,TRAIN_026,1,0.531992,2022-06-19 9:20,T100306,T_31,2.0,95.0,54.0,477.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,TRAIN_030,1,0.528646,2022-06-19 23:39,T100306,T_31,2.0,88.0,51.0,476.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,TRAIN_036,1,0.529633,2022-06-22 3:39,T100306,T_31,2.0,91.0,60.0,478.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,TRAIN_042,1,0.530965,2022-06-23 3:39,T100306,T_31,2.0,95.0,51.0,476.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,TRAIN_043,1,0.529498,2022-06-23 6:28,T100306,T_31,2.0,95.0,57.0,477.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,0,0.523386,NaN,T100306,NaN,2.0,87.0,50.0,466.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,0,0.525067,NaN,T100306,NaN,2.0,87.0,50.0,511.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,0,0.524327,NaN,T100306,NaN,2.0,95.0,53.0,499.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,0,0.524737,NaN,T100306,NaN,2.0,93.0,50.0,466.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
trainT_31 = pd.concat([train_T100304_T,train_T100306_T]).sort_values(by='PRODUCT_ID')
trainT_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_11,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
25,TRAIN_025,1,0.529362,2022-06-19 9:11,T100304,T_31,2.0,97.0,11.0,517.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,TRAIN_026,1,0.531992,2022-06-19 9:20,T100306,T_31,2.0,95.0,10.0,477.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,TRAIN_029,1,0.532405,2022-06-19 23:31,T100304,T_31,2.0,100.0,11.0,516.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,TRAIN_030,1,0.528646,2022-06-19 23:39,T100306,T_31,2.0,88.0,10.0,476.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,TRAIN_036,1,0.529633,2022-06-22 3:39,T100306,T_31,2.0,91.0,10.0,478.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,0,0.523386,NaN,T100306,NaN,2.0,87.0,NaN,466.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,0,0.525067,NaN,T100306,NaN,2.0,87.0,NaN,511.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,0,0.524327,NaN,T100306,NaN,2.0,95.0,NaN,499.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,0,0.524737,NaN,T100306,NaN,2.0,93.0,NaN,466.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


PRODUCT_CODE O

In [36]:
train_T100304_O = train_T100304[train_T100304['PRODUCT_CODE'] == 'O_31']

train_T100304_O_0 = train_T100304_O[train_T100304_O['Y_Class'] == 0]
# train_T100304_O_0 = train_T100304_O_0.fillna(train_T100304_O_0.max() * -1)

train_T100304_O_1 = train_T100304_O[train_T100304_O['Y_Class'] == 1]
train_T100304_O_1 = train_T100304_O_1.fillna(train_T100304_O_1.max() * -1)

train_T100304_O_2 = train_T100304_O[train_T100304_O['Y_Class'] == 2]
# train_T100304_O_2 = train_T100304_O_2.fillna(train_T100304_O_2.max() * -1)

train_T100304_O = pd.concat([train_T100304_O_0,train_T100304_O_1,train_T100304_O_2]).sort_values(by='PRODUCT_ID')

In [37]:
train_T100306_O = train_T100306[train_T100306['PRODUCT_CODE'] == 'O_31']

train_T100306_O_0 = train_T100306_O[train_T100306_O['Y_Class'] == 0]
# train_T100306_O_0 = train_T100306_O_0.fillna(train_T100306_O_0.max() * -1)

train_T100306_O_1 = train_T100306_O[train_T100306_O['Y_Class'] == 1]
train_T100306_O_1 = train_T100306_O_1.fillna(train_T100306_O_1.max() * -1)

train_T100306_O_2 = train_T100306_O[train_T100306_O['Y_Class'] == 2]
train_T100306_O_2 = train_T100306_O_2.fillna(train_T100306_O_2.max() * -1)

train_T100306_O = pd.concat([train_T100306_O_0,train_T100306_O_1,train_T100306_O_2]).sort_values(by='PRODUCT_ID')

In [38]:
trainO_31 = pd.concat([train_T100304_O,train_T100306_O]).sort_values(by='PRODUCT_ID')
trainO_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
569,TRAIN_569,1,0.530533,2022-09-03 18:32,T100304,O_31,4.0,98.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,TRAIN_570,2,0.534951,2022-09-03 18:40,T100306,O_31,6.0,90.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,TRAIN_571,1,0.525916,2022-09-03 18:48,T100304,O_31,4.0,100.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,TRAIN_572,2,0.535205,2022-09-03 18:56,T100306,O_31,6.0,89.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,TRAIN_597,1,0.533702,2022-09-08 14:46,T100306,O_31,21.0,87.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
testA_31 = test_df[test_df['PRODUCT_CODE'] == 'A_31']
testT_31 = test_df[test_df['PRODUCT_CODE'] == 'T_31']
testO_31 = test_df[test_df['PRODUCT_CODE'] == 'O_31']

# product code별 학습진행
  - LINE 별 dataset 에서 Y_Class별로 나누어 mean 적용(결측치 없앰)
  - 나머지 존재하는 NaN에 대해서는 -1 대체 (변수가 의미없음을 의미)
  - 변수개수는 줄이지 않았음
  - PRODUCT_CODE O는 6개 존재(Y_Class = 1 4개, Y_Class = 2 2개) -> 이는 분류 보단 회귀 이후 라벨링이 적절해보임.

Dataset:  
- trainA_31  
- trainT_31  
- trainO_31

## test nan 처리에 대해 고민해봐야할듯
- test에도 O는 4개밖에 없어서 O를 맞추는 거는 크게 고민 안해도 될듯
- A와 T위주로 맞추는걸로

In [40]:
trainA_31 = remove_all_nan(trainA_31)
trainT_31 = remove_all_nan(trainT_31)
trainO_31 = remove_all_nan(trainO_31)

In [41]:
trainA_31 = remove_one_value(trainA_31)
trainT_31 = remove_one_value(trainT_31)
trainO_31 = remove_one_value(trainO_31)

In [42]:
trainA_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,X_128,X_129,X_132,X_133,X_134,...,X_2812,X_2815,X_2816,X_2819,X_2820,X_2823,X_2824,X_2826,X_2827,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,7813.0,7813.0,0.19,0.20,0.19,...,51.4,55.5,55.5,55.2,55.2,55.2,55.2,52.6,52.6,-1.0
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,NaN,NaN,0.20,0.21,0.20,...,53.8,60.7,60.7,59.2,59.2,61.0,61.0,53.8,53.8,-1.0
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,7815.0,7815.0,0.19,0.20,0.19,...,51.4,55.5,55.5,55.2,55.2,55.2,55.2,52.6,52.6,-1.0
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,NaN,NaN,0.20,0.21,0.20,...,53.8,60.7,60.7,59.2,59.2,61.0,61.0,53.8,53.8,-1.0
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,7817.0,7817.0,0.19,0.20,0.18,...,51.4,55.5,55.5,55.2,55.2,55.2,55.2,52.6,52.6,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NaN,0,0.523173,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,0,0.500856,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,0,0.521249,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,NaN,0,0.518024,NaN,T010305,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality'])
trainT_31_x = trainT_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality'])
trainO_31_x = trainO_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality'])

In [44]:
trainA_31_x = pd.get_dummies(trainA_31_x)
trainT_31_x = pd.get_dummies(trainT_31_x)
trainO_31_x = pd.get_dummies(trainO_31_x)

In [45]:
trainA_31_x

,X_128,X_129,X_132,X_133,X_134,X_136,X_137,X_138,X_139,X_140,...,X_2820,X_2823,X_2824,X_2826,X_2827,X_2871,LINE_T010305,LINE_T010306,LINE_T050304,LINE_T050307
0,7813.0,7813.0,0.19,0.20,0.19,228.0,228.0,225.0,582.9,587.1,...,55.2,55.2,55.2,52.6,52.6,-1.0,0,0,1,0
1,NaN,NaN,0.20,0.21,0.20,413.0,414.0,414.0,589.3,595.8,...,59.2,61.0,61.0,53.8,53.8,-1.0,0,0,0,1
2,7815.0,7815.0,0.19,0.20,0.19,228.0,228.0,225.0,583.8,587.6,...,55.2,55.2,55.2,52.6,52.6,-1.0,0,0,1,0
3,NaN,NaN,0.20,0.21,0.20,414.0,414.0,414.0,589.8,596.1,...,59.2,61.0,61.0,53.8,53.8,-1.0,0,0,0,1
4,7817.0,7817.0,0.19,0.20,0.18,228.0,228.0,225.0,583.2,587.3,...,55.2,55.2,55.2,52.6,52.6,-1.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0


Dataset:  
- trainA_31  
- trainT_31  
- trainO_31  

PRODUCT_CODE 별 존재하는 NaN에 대해서는 iterativeimputer 적용
  - 모두 NaN인 칼럼은 삭제(train)
  - test는 train 칼럼 그대로 이용

test 결측치
  - train에 iterativeimputer fit, test에 transform 적용

Dataset:
- trainA_31_x, testA_31_x
- trainT_31_x, testT_31_x 
- trainO_31_x, testO_31_x




In [46]:
testA_31_x = pd.concat([testA_31[trainA_31_x.columns[:-4]], testA_31['LINE']], axis=1)
testA_31_x = pd.get_dummies(testA_31_x)

testT_31_x = pd.concat([testT_31[trainT_31_x.columns[:-2]], testT_31['LINE']], axis=1)
testT_31_x = pd.get_dummies(testT_31_x)

testO_31_x = pd.concat([testO_31[trainO_31_x.columns[:-2]], testO_31['LINE']], axis=1)
testO_31_x = pd.get_dummies(testO_31_x)

In [47]:
trainA_31_x=trainA_31_x.fillna(-1)
testA_31_x=testA_31_x.fillna(-1)
train_T_31_x=trainT_31_x.fillna(-1)
test_T_31_x=testT_31_x.fillna(-1)
train_O_31_x=trainO_31_x.fillna(-1)
test_O_31_x=testO_31_x.fillna(-1)

In [48]:
# classification
trainA_31_y_c = trainA_31['Y_Class']
trainT_31_y_c = trainT_31['Y_Class']
trainO_31_y_c = trainO_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
trainT_31_y_r = trainT_31['Y_Quality']
trainO_31_y_r = trainO_31['Y_Quality']

## model

In [49]:
!pip install --target=$my_path catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.9 MB/s eta 0:00:00


In [50]:
from catboost import *

In [51]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0080918	total: 173ms	remaining: 4m 19s
500:	learn: 0.0015470	total: 1m 1s	remaining: 2m 1s
1000:	learn: 0.0004394	total: 1m 54s	remaining: 57s
1499:	learn: 0.0001308	total: 2m 46s	remaining: 0us


In [52]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(trainT_31_x, trainT_31_y_r)
pred_t = model.predict(testT_31_x)

0:	learn: 0.0066346	total: 43ms	remaining: 1m 4s
500:	learn: 0.0010170	total: 28.9s	remaining: 57.7s
1000:	learn: 0.0002236	total: 46.9s	remaining: 23.4s
1499:	learn: 0.0000531	total: 1m 12s	remaining: 0us


In [53]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(trainO_31_x, trainO_31_y_r)
pred_o = model.predict(testO_31_x)

0:	learn: 0.0031702	total: 16ms	remaining: 23.9s
500:	learn: 0.0000480	total: 3.63s	remaining: 7.23s
1000:	learn: 0.0000008	total: 6.77s	remaining: 3.37s
1499:	learn: 0.0000000	total: 8.12s	remaining: 0us


In [54]:
testA_31['Y_quanlity'] = pred_a
testT_31['Y_quanlity'] = pred_t
testO_31['Y_quanlity'] = pred_o

<ipython-input-54-30f484713f3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testA_31['Y_quanlity'] = pred_a
<ipython-input-54-30f484713f3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testT_31['Y_quanlity'] = pred_t
<ipython-input-54-30f484713f3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [55]:
testA_31['Y_Class'] = 1
testT_31['Y_Class'] = 1
testO_31['Y_Class'] = 1

<ipython-input-55-dd33eabf9461>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testA_31['Y_Class'] = 1
<ipython-input-55-dd33eabf9461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testT_31['Y_Class'] = 1
<ipython-input-55-dd33eabf9461>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [56]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

testT_31.loc[(testT_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testT_31.loc[(testT_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

testO_31.loc[(testO_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testO_31.loc[(testO_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [57]:
submita = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')

In [58]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],testT_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],testO_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

In [59]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('cat + mice.csv',index=False)

## 검증

In [61]:
result = pd.read_csv('/content/cat + mice.csv')

In [62]:
result['Y_Class'].value_counts()

1    306
2      4
Name: Y_Class, dtype: int64

In [63]:
trainA_31.to_csv('/content/trainA_31')
trainT_31.to_csv('/content/trainT_31')
trainO_31.to_csv('/content/trainO_31')

In [ ]:
cat2['Y_Class'].value_counts()

In [ ]:
from collections import Counter
Counter(result['Y_Class'] != cat2['Y_Class'])

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [ ]:
result_0 = result[result['Y_Class']==0]
cat_0 = cat2[cat2['Y_Class']==0]

In [ ]:
result_0.index

In [ ]:
cat_0.index

In [ ]:
sum([1 for i in cat_0.index if i in result_0.index])

In [ ]:
result_2 = result[result['Y_Class']==2]
cat_2 = cat2[cat2['Y_Class']==2]
sum([1 for i in cat_2.index if i in result_2.index])

In [ ]:
from collections import Counter
Counter(result['Y_Class'] != cat2['Y_Class'])